In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../cvxportfolio'))

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import quandl

import cvxportfolio as cp

In [3]:
quandl.ApiConfig.api_key = '8wnB7EcvJm1xEvJjzWBR'

### Download data 

In [4]:
tickers = ['AMZN', 'GOOGL', 'MCD', 'NKE']
start_date='2016-01-01'
end_date='2017-03-13'
prices = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Close'])
                for ticker in tickers]))

returns=prices.pct_change()
returns[["USDOLLAR"]]=quandl.get('FRED/DTB3', start_date=start_date, end_date=end_date)/(250*100)
returns = returns.fillna(method='ffill').iloc[1:]

In [5]:
r_hat = returns.rolling(window=250, min_periods=250).mean().shift(1).dropna()
Sigma_hat = returns.rolling(window=250, min_periods=250).cov().unstack().shift(1).stack()

In [6]:
Sigma_hat

AMZN         GOOGL           MCD           NKE  \
Date                                                                          
2016-12-30 AMZN      3.375380e-04  1.385427e-04  5.457935e-05  8.138472e-05   
           GOOGL     1.385427e-04  1.589377e-04  4.715114e-05  8.499972e-05   
           MCD       5.457935e-05  4.715114e-05  9.191701e-05  4.942811e-05   
           NKE       8.138472e-05  8.499972e-05  4.942811e-05  1.991368e-04   
           USDOLLAR -2.618391e-09  1.995129e-09  2.667937e-09  1.694502e-09   
2017-01-03 AMZN      3.391428e-04  1.397114e-04  5.565264e-05  8.205454e-05   
           GOOGL     1.397114e-04  1.596192e-04  4.749664e-05  8.506013e-05   
           MCD       5.565264e-05  4.749664e-05  9.151305e-05  4.877680e-05   
           NKE       8.205454e-05  8.506013e-05  4.877680e-05  1.983391e-04   
           USDOLLAR -3.341861e-09  1.651918e-09  2.656464e-09  1.855445e-09   
2017-01-04 AMZN      3.391858e-04  1.400050e-04  5.528289e-05  8.230176e-05   
           GOOGL     1.400050e-04  1.610827e-04  4.604952e-05  8.669074e-05   
           MCD       5.528289e-05  4.604952e-05  9.254702e-05  4.676526e-05   
           NKE       8.230176e-05  8.669074e-05  4.676526e-05  1.997547e-04   
           USDOLLAR -3.244700e-09  2.255027e-09  1.941058e-09  2.407767e-09   
2017-01-05 AMZN      3.328106e-04  1.360552e-04  5.150227e-05  7.840286e-05   
           GOOGL     1.360552e-04  1.586706e-04  4.374995e-05  8.404909e-05   
           MCD       5.150227e-05  4.374995e-05  9.035714e-05  4.417748e-05   
           NKE       7.840286e-05  8.404909e-05  4.417748e-05  1.988282e-04   
           USDOLLAR -3.893774e-09  1.762606e-09  1.446054e-09  2.593105e-09   
2017-01-06 AMZN      3.363085e-04  1.366277e-04  5.166603e-05  7.825035e-05   
           GOOGL     1.366277e-04  1.580205e-04  4.368205e-05  8.314291e-05   
           MCD       5.166603e-05  4.368205e-05  9.035275e-05  4.405815e-05   
           NKE       7.825035e-05  8.314291e-05  4.405815e-05  1.977908e-04   
           USDOLLAR -3.043186e-09  1.672823e-09  1.456721e-09  2.284711e-09   
2017-01-09 AMZN      3.366336e-04  1.375559e-04  5.163798e-05  7.869576e-05   
           GOOGL     1.375559e-04  1.588371e-04  4.407953e-05  8.397354e-05   
           MCD       5.163798e-05  4.407953e-05  9.023077e-05  4.413460e-05   
           NKE       7.869576e-05  8.397354e-05  4.413460e-05  1.982950e-04   
           USDOLLAR -2.151637e-09  2.176507e-09  1.911629e-09  3.015036e-09   
...                           ...           ...           ...           ...   
2017-03-06 AMZN      1.949340e-04  8.086907e-05  2.145621e-05  3.599387e-05   
           GOOGL     8.086907e-05  1.106736e-04  1.868243e-05  4.779977e-05   
           MCD       2.145621e-05  1.868243e-05  7.062079e-05  2.334669e-05   
           NKE       3.599387e-05  4.779977e-05  2.334669e-05  1.535858e-04   
           USDOLLAR -3.616420e-09  1.247801e-09  2.870684e-09  4.249985e-09   
2017-03-07 AMZN      1.949023e-04  8.094072e-05  2.170464e-05  3.620395e-05   
           GOOGL     8.094072e-05  1.107066e-04  1.866488e-05  4.777420e-05   
           MCD       2.170464e-05  1.866488e-05  7.019143e-05  2.295370e-05   
           NKE       3.620395e-05  4.777420e-05  2.295370e-05  1.532288e-04   
           USDOLLAR -4.019167e-09  1.078647e-09  3.015390e-09  4.431713e-09   
2017-03-08 AMZN      1.948863e-04  8.110497e-05  2.183105e-05  3.592809e-05   
           GOOGL     8.110497e-05  1.097472e-04  1.803229e-05  4.929529e-05   
           MCD       2.183105e-05  1.803229e-05  6.980150e-05  2.393346e-05   
           NKE       3.592809e-05  4.929529e-05  2.393346e-05  1.508369e-04   
           USDOLLAR -4.202919e-09  1.473649e-09  3.101536e-09  3.954139e-09   
2017-03-09 AMZN      1.949064e-04  8.123146e-05  2.185379e-05  3.602191e-05   
           GOOGL     8.123146e-05  1.094678e-04  1.794597e-05  4.896134e-05   
           MCD       2.185379e-05  1.794597e-05  6.977828e-05  2.384239e-05   


In [7]:
tcost_model=cp.TcostModel(half_spread=10E-4)
hcost_model=cp.HcostModel(borrow_costs=1E-4)

risk_model = cp.FullSigma(Sigma_hat)
gamma_risk, gamma_trade, gamma_hold = 5., 1., 1.
leverage_limit = cp.LeverageLimit(3)

spo_policy = cp.SinglePeriodOpt(return_forecast=r_hat, 
                                costs=[gamma_risk*risk_model, gamma_trade*tcost_model, gamma_hold*hcost_model],
                                constraints=[leverage_limit])

### Compute optimal trades 

In [8]:
current_portfolio=pd.Series(index=r_hat.columns,data=0)
current_portfolio.USDOLLAR=10000

shares_to_trade=spo_policy.get_rounded_trades(current_portfolio, prices, t=end_date)
shares_to_trade

AMZN     4.0
GOOGL    0.0
MCD      0.0
NKE      0.0
dtype: float64

In [9]:
spo_policy.get_trades(current_portfolio, t=end_date)

AMZN        3452.933557
GOOGL          0.000037
MCD            0.000034
NKE            0.000019
USDOLLAR   -3452.933647
dtype: float64

#### Export to Excel file 

In [10]:
pd.DataFrame({pd.datetime.today().date().__str__():shares_to_trade}).to_excel('shares_to_trade.xls')